In [1]:
import ray

In [2]:
ray.init()

2024-02-26 17:04:10,561	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.11
Ray version:,2.9.2
Dashboard:,http://127.0.0.1:8265


In [3]:
from dataclasses import dataclass

import gymnasium as gym


@ray.remote
@dataclass
class RemoteEnv():
    env: gym.Env

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        return self.env.reset()


In [12]:
from typing import Any


@ray.remote
@dataclass
class RemoteAgent():
    agent: Any

    def predict(self, state):
        return self.agent.predict(state)

    def learn(self, state, action, reward, next_state, done):
        return self.agent.learn(state, action, reward, next_state, done)

    def train(self):
        return self.agent.train()


In [13]:
from stable_baselines3 import ppo

env = gym.make('CartPole-v0')
remote_env = RemoteEnv.remote(env)

agent = ppo.PPO("MlpPolicy", env)
remote_agent = RemoteAgent.remote(agent)

In [38]:
import stable_baselines3


def training(env, agent):
    env: gym.Env = env
    agent: stable_baselines3.PPO = agent
    for _ in range(1000):
        state, info = env.reset()
        done = False
        while not done:
            action, states = agent.predict(state)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            agent.learn((state, action, reward, next_state, done))
            state = next_state
        agent.train()

In [39]:
training(env, agent)

TypeError: '<' not supported between instances of 'int' and 'tuple'

In [26]:
@ray.remote
def remote_training():
    env = remote_env
    agent = remote_agent

    for _ in range(1):
        state, info = ray.get(env.reset.remote())
        done = False
        while not done:
            action = ray.get(agent.predict.remote(state))
            next_state, reward, terminated, truncated, _ = ray.get(env.step.remote(action))
            done = terminated or truncated
            ray.get(agent.learn.remote(state, action, reward, next_state, done))
            state = next_state
        ray.get(agent.train.remote())

In [27]:
ray.get(remote_training.remote())

RayTaskError(AssertionError): [36mray::remote_training()[39m (pid=13392, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1813, in ray._raylet.execute_task
  File "C:\Users\kaosl\AppData\Local\Temp\ipykernel_5288\3594627384.py", line 11, in remote_training
  File "c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-packages\ray\_private\auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-packages\ray\_private\worker.py", line 2624, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(AssertionError): [36mray::RemoteEnv.step()[39m (pid=15600, ip=127.0.0.1, actor_id=6a6b2f185c6cd650702af75f01000000, repr=RemoteEn

In [14]:
from tqdm import tqdm

state, info = ray.get(remote_env.reset.remote())
for step in tqdm(range(200)):
    action, states = ray.get(remote_agent.predict.remote(state))
    next_state, reward, terminated, truncated, info = ray.get(remote_env.step.remote(action))
    done = terminated or truncated
    ray.get(remote_agent.learn.remote(state, action, reward, next_state, done))
    state = next_state
    if done:
        state, info = ray.get(remote_env.reset.remote())

  0%|          | 0/200 [00:02<?, ?it/s]


RayTaskError(ValueError): [36mray::RemoteAgent.learn()[39m (pid=2548, ip=127.0.0.1, actor_id=62e9447c4c8f73a7d563795901000000, repr=RemoteAgent(agent=<stable_baselines3.ppo.ppo.PPO object at 0x00000209BC59A680>))
  File "python\ray\_raylet.pyx", line 1813, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1754, in ray._raylet.execute_task.function_executor
  File "c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-packages\ray\util\tracing\tracing_helper.py", line 467, in _resume_span
    return method(self, *_args, **_kwargs)
  File "C:\Users\kaosl\AppData\Local\Temp\ipykernel_5288\1468660551.py", line 13, in learn
  File "c:\Users\kaosl\AppData\Local\pypoetry\Cache\virtualenvs\pokerl-qa45OdQw-py3.10\lib\site-pa